In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
import ast
import datetime
import json
import pathlib
import pandas as pd
from pprint import pprint
import requests

cwd = pathlib.Path().cwd()
proj_dir = cwd.parent.parent
secrets_dir = proj_dir / 'secrets'
planning_dir = proj_dir / 'planning'

In [ ]:
schema_df = pd.read_csv(planning_dir / 'schema_plan.tsv',  delimiter='\t')
attrs_lst = schema_df['FIELD_NAME'].to_list() # at 20250317 service layer at idx 0 includes every field in the schema plan
attrs = {attr: None for attr in attrs_lst}

In [ ]:
## non-nullable fields

attrs['FireActivityStatus'] = 'Out'

attrs['AnalysisBufferMiles'] = 0
attrs['ReportedAcOverPerimAc'] = 0
attrs['HasError'] = 0

attrs['DefaultLabel'] = 'TEST'
attrs['ReportedAcres'] = 5


attrs['wfigs_IrwinID'] = '{D0160365-fake-fire-haha-A8AD445EA18A}'

date_str = "03/1/2025 12:00:00"
attrs['wfigs_ModifiedOnDateTime_dt'] = date_str

In [ ]:
with open (cwd.parent / 'dummy_fire_geom.json', 'r') as file:
    dummy_geom = json.load(file)

init_feat = [
    {
        "attributes": attrs,
        "geometry": dummy_geom['geometry']
    }
]

In [ ]:
pprint(init_feat, indent=4)

In [ ]:
## retrieve token

with open(secrets_dir / 'nifc_ago_credentials.txt', 'r') as file:
    nifc_credentials_str = file.read()
    nifc_credentials_tup = ast.literal_eval(nifc_credentials_str)

nifc_gis = GIS(*nifc_credentials_tup)

token = nifc_gis._con.token

In [ ]:
post_data = {
    'adds': json.dumps(init_feat),
    'rollbackOnFailure': True,
    'f': 'json',
    'token': token,
}

In [ ]:
response = requests.post(
    r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/AK_Wildfire_Values_at_Risk/FeatureServer/0/applyEdits',
    data=post_data
)

In [ ]:
response.json()